In [1]:
!python --version
!pip list | grep tensorflow

Python 3.6.12 :: Anaconda, Inc.
tensorflow           1.15.4
tensorflow-estimator 1.15.1


In [2]:


import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN, LSTM, Input, Bidirectional
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist



## Clean Data

In [3]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# resize and normalize
#image_size = x_train.shape[1]
#print(x_train.shape)

x_train = np.reshape(x_train,[x_train.shape[0], -1, 1])
x_test = np.reshape(x_test,[x_test.shape[0], -1, 1])

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print("x_train", x_train.shape)
print("y_train", y_train.shape)

print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (60000, 784, 1)
y_train (60000, 10)
x_test (10000, 784, 1)
y_test (10000, 10)


## Setup Model

In [4]:
# network parameters
input_shape = (x_train.shape[1:])
batch_size = 128
units = 100
dropout = 0.00

# model is RNN with 256 units, input is 28-dim vector 28 timesteps
model = Sequential()
model.add(Input(input_shape, name="input"))
model.add(Bidirectional(LSTM(units=units,
                    dropout=dropout, name="ltsm" )))
                    #input_shape=input_shape))
model.add(Dense(num_labels, name="dense"))
model.add(Activation('softmax', name="activate"))
model.summary()
print("units", units)
print("input_shape", input_shape)
print("num_labels", num_labels)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 200)               81600     
_________________________________________________________________
dense (Dense)                (None, 10)                2010      
_________________________________________________________________
activate (Activation)        (None, 10)                0         
Total params: 83,610
Trainable params: 83,610
Non-trainable params: 0
_____________

## Model compile and fit (train)

In [15]:
# enable this if pydot can be installed
# pip install pydot
#plot_model(model, to_file='rnn-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer
# accuracy is good metric for classification tasks

from tensorflow.keras.optimizers import RMSprop

learning_rate = 1e-3
rmsprop = RMSprop(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer= rmsprop,
              metrics=['accuracy'])

#model.compile(loss='categorical_crossentropy',
#              optimizer='sgd',
#              metrics=['accuracy'])
# train the network

model.fit(x_train, y_train, 
          epochs=200, 
          batch_size=batch_size,
          validation_data=(x_test, y_test))



ModuleNotFoundError: No module named 'tensor'

## Model pred

In [ ]:
_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))